In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm 


In [2]:
# Initialize an empty set to store unique genres
unique_genres = set()

# Assuming your dataset file is named 'your_dataset.txt'
with open(r'C:\Users\GDIT\Desktop\GIKI\Hamza\CodSoft\Movie Genre Classification\Genre Classification Dataset\train_data.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # Split the line by ':::' to extract the genre part
        parts = line.strip().split(':::')
        if len(parts) >= 3:
            genres_list = parts[2].strip().split(',')  # Assuming genres are separated by commas
            unique_genres.update(genres_list)

# Print the unique genres
print("Unique genres in the dataset:", unique_genres)


Unique genres in the dataset: {'sport', 'thriller', 'sci-fi', 'music', 'history', 'musical', 'adult', 'reality-tv', 'short', 'comedy', 'drama', 'family', 'documentary', 'crime', 'fantasy', 'western', 'animation', 'war', 'biography', 'romance', 'adventure', 'mystery', 'action', 'news', 'game-show', 'talk-show', 'horror'}


In [3]:
# define a fallback genre for which the model finds hard to predict
fallback_genre = 'Unknown'

In [6]:
# Load the training data set 
try:
    with tqdm(total=50, desc="Loading Train Data") as pbar:
        train_data = pd.read_csv(r'C:\Users\GDIT\Desktop\GIKI\Hamza\CodSoft\Movie Genre Classification\Genre Classification Dataset\train_data.txt', sep=':::', header=None, names=['SerialNumber', 'MOVIE_NAME', 'GENRE', 'MOVIE_PLOT'], engine='python')
        pbar.update(50)
except Exception as e:
    print(f"Error loading train_data: {e}")
    raise


Loading Train Data: 100%|██████████| 50/50 [00:00<00:00, 115.71it/s]


In [7]:
# data processing for training the data and testing
X_train = train_data['MOVIE_PLOT'].astype(str).apply(lambda doc: doc.lower())
genre_labels = [genre.split(', ') for genre in train_data['GENRE']]
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(genre_labels)

In [9]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

In [10]:
# Fit and transform the training data with progress bar
with tqdm(total=50, desc="Vectorizing Training Data") as pbar:
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    pbar.update(50)

Vectorizing Training Data:   0%|          | 0/50 [00:00<?, ?it/s]

Vectorizing Training Data: 100%|██████████| 50/50 [00:06<00:00,  7.56it/s]


In [18]:
# Train a MultiOutput Naive Bayes classifier using the training data
with tqdm(total=50, desc="Training Model") as pbar:
    naive_bayes = MultinomialNB()
    multi_output_classifier = MultiOutputClassifier(naive_bayes)
    multi_output_classifier.fit(X_train_tfidf, y_train)
    pbar.update(50)

Training Model: 100%|██████████| 50/50 [00:00<00:00, 53.26it/s]


In [12]:
# Load test dataset from test_data.txt
try:
    with tqdm(total=50, desc="Loading Test Data") as pbar:
        test_data = pd.read_csv(r'C:\Users\GDIT\Desktop\GIKI\Hamza\CodSoft\Movie Genre Classification\Genre Classification Dataset\test_data.txt', sep=':::', header=None, names=['SerialNumber', 'MOVIE_NAME', 'MOVIE_PLOT'], engine='python')
        pbar.update(50)
except Exception as e:
    print(f"Error loading test_data: {e}")
    raise

Loading Test Data: 100%|██████████| 50/50 [00:00<00:00, 91.06it/s]


In [13]:
# Data preprocessing for test data
X_test = test_data['MOVIE_PLOT'].astype(str).apply(lambda doc: doc.lower())

In [14]:
# Transform the test data with progress bar
with tqdm(total=50, desc="Vectorizing Test Data") as pbar:
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    pbar.update(50)

Vectorizing Test Data: 100%|██████████| 50/50 [00:06<00:00,  8.15it/s]


In [17]:
# Predict genres on the test data
with tqdm(total=50, desc="Predicting on Test Data") as pbar:
    y_pred = multi_output_classifier.predict(X_test_tfidf)
    pbar.update(50)

Predicting on Test Data: 100%|██████████| 50/50 [00:00<00:00, 72.90it/s]


In [19]:
# Create a DataFrame for test data with movie names and predicted genres
test_movie_names = test_data['MOVIE_NAME']
predicted_genres = mlb.inverse_transform(y_pred)
test_results = pd.DataFrame({'MOVIE_NAME': test_movie_names, 'PREDICTED_GENRES': predicted_genres})

In [20]:
# Replace empty unpredicted genres with the fallback genre
test_results['PREDICTED_GENRES'] = test_results['PREDICTED_GENRES'].apply(lambda genres: [fallback_genre] if len(genres) == 0 else genres)

In [21]:
# Write the results to an output text file with proper formatting
with open("model_evaluation.txt", "w", encoding="utf-8") as output_file:
    for _, row in test_results.iterrows():
        movie_name = row['MOVIE_NAME']
        genre_str = ', '.join(row['PREDICTED_GENRES'])
        output_file.write(f"{movie_name} ::: {genre_str}\n")

In [22]:
# Calculate evaluation metrics using training labels (as a proxy)
y_train_pred = multi_output_classifier.predict(X_train_tfidf)

In [23]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_train, y_train_pred)
precision = precision_score(y_train, y_train_pred, average='micro')
recall = recall_score(y_train, y_train_pred, average='micro')
f1 = f1_score(y_train, y_train_pred, average='micro')

In [24]:
# Append the evaluation metrics to the output file
with open("model_evaluation.txt", "a", encoding="utf-8") as output_file:
    output_file.write("\n\nModel Evaluation Metrics:\n")
    output_file.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    output_file.write(f"Precision: {precision:.2f}\n")
    output_file.write(f"Recall: {recall:.2f}\n")
    output_file.write(f"F1-score: {f1:.2f}\n")

In [25]:
print("Model evaluation results and metrics have been saved to 'model_evaluation.txt'.")

Model evaluation results and metrics have been saved to 'model_evaluation.txt'.
